<h1>flask_sqlalchemy</h1><br>

<div>SQL Alchemy is a popular Object Relational Mapper (ORM). We use it in this project for the following reasons:</div>
<br>
<ul><li>it creates a bridge from OOP code to relational DB structure by mapping objects to relations, thus allowing access to DB in object oriented way</li><br>
    <li>it is easy to use for several DBs and allows seamless transition from one DB to another. this is very convenient for development, as we can use local easy to set up SQLite.</li><br>
    <li>seamless representation of DB structure as classes in python.</li>
    </ul><br>
<div>Important learnings during the Project are documented below. This shall not be interpreted as full documentation of flask_sqlalchemy module. Future learnings relevant to SQL alchemy shall be appended.  
  

To display the functionality of flask_sqlalchemy, we need a test app first:

In [1]:
from flask import Flask
test=Flask('Test')

SQLite offer lightweight DBs. They are easy to set up, making them a suitable choice for development:

In [2]:
test.config['SQLALCHEMY_DATABASE_URI']='sqlite:///calendar.db' #triple slash for local URI

In order to use SQL Alchemy, we need to set it up first:

In [3]:
from flask_sqlalchemy import SQLAlchemy

In [4]:
from models import User,Appointment,db

In [5]:
db.init_app(test)

c:\python38\lib\site-packages\flask_sqlalchemy\__init__.py:833: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [6]:
db.create_all(app=test)

In [7]:
import secrets
import datetime

In [9]:
start=datetime.datetime(year=datetime.MINYEAR,month=1,day=1,hour=9,minute=30)
end=datetime.datetime(year=datetime.MINYEAR,month=1,day=1,hour=19,minute=0)

In [10]:
test_user1=User(name="jakob",secret=secrets.token_hex(nbytes=4),avail_days="yyyyy",avail_start=start,avail_end=end,buffer=10)

In [11]:
test_user2=User(name="sofia",secret=secrets.token_hex(nbytes=4),avail_days="yyyyn",avail_start=start,avail_end=end,buffer=10)

In [12]:
with test.app_context():
    db.session.add(test_user1)
    db.session.add(test_user2)
    db.session.commit()

After running the line above, local DB should be in directory. The DB now contains the schema specified in our class model.

Now, we can create instances of our class model:

In [ ]:
sev=Project_member(name='Sev',role='Product Owner')
jakob=Project_member(name='Jakob',role='Dev')

In order to add them to our DB, we need to add them to our DB session and commit:

In [ ]:
db.session.add(sev)
db.session.add(jakob)
db.session.commit()

Our project members should now be featured in the DB. We can check this with some queries:

In [ ]:
Project_member.query.all()

In [ ]:
sev=Project_member.query.first()
sev

Note that we can also access attributes that aren't in our representation operation with the dot notation:

In [ ]:
sev.id

Knowing that Sev's PK is 1, we could have also fetched him by specifying a PK in the command:

In [ ]:
Project_member.query.get(1)

We can also filter our query results:

In [ ]:
Project_member.query.filter_by(role='Dev').all()

If we want to wipe our DB, we can do so:

In [ ]:
db.drop_all()

In [ ]:
#Project_member.query.all() #now causes error b/c table does not exist

So to recap, we get to our previous state by creating the DB schema and committing user model instances to it:

In [ ]:
db.create_all()
sev=Project_member(name='Sev',role='Product Owner')
jakob=Project_member(name='Jakob',role='Dev')
db.session.add(sev)
db.session.add(jakob)
db.session.commit()

In [ ]:
Project_member.query.all()

We might want to add a class model wih tasks:

In [ ]:
from datetime import datetime #need this for the time representation

class Task(db.Model):
    id=db.Column(db.Integer,primary_key=True)
    name=db.Column(db.String(100),nullable=False)
    issued=db.Column(db.DateTime,nullable=False,default=datetime.utcnow) #now parentheses for utcnow to pass function as argument
    description=db.Column(db.Text)
    
    def __repr__(self):
        return f"Task('{self.name}', '{self.issued}')"

In [ ]:
db.create_all()

In [ ]:
task1=Task(name='Documentation')
db.session.add(task1)
db.session.commit()
Task.query.all()

If we want to link the Tasks to Project Members, we have to extend our Class Models to include foreign keys:

In [ ]:
db.drop_all() #Let's clear our DB once more.
db.engine.dispose() #Let's get rid of all our current connections to the DB
db=SQLAlchemy(test) #let's start with a fresh SQLAlchemy instance as we had already established class models in the previous

class Project_member(db.Model):
    id=db.Column(db.Integer,primary_key=True) #specifies id as PK
    name=db.Column(db.String(40),unique=True,nullable=False) #name must be unique and must contain data
    role=db.Column(db.String(30))
    
    """THIS IS THE NEW CODE"""
    tasks=db.relationship('Task',backref='owner',lazy=True) #lazy arguments defines how data is loaded. set to True, sqlalchemy loads the data in one go.
    """END OF NEW CODE"""
    
    def __repr__(self): #function specifies how data is represented when queried.
        return f"Project Member('{self.name}', '{self.role}')"
    
class Task(db.Model):
    id=db.Column(db.Integer,primary_key=True)
    name=db.Column(db.String(100),nullable=False)
    issued=db.Column(db.DateTime,nullable=False,default=datetime.utcnow) #now parentheses for utcnow to pass function as argument
    description=db.Column(db.Text)
    
    """THIS IS MORE NEW CODE"""
    owner_id=db.Column(db.Integer,db.ForeignKey('project_member.id'),nullable=False) #note the lower case for the foreign key as it refers to the table name, not our class model.
    """END OF NEW CODE"""
    
    def __repr__(self):
        return f"Task('{self.name}', '{self.issued}')"

In [ ]:
db.create_all()
sev=Project_member(name='Sev',role='Product Owner')
jakob=Project_member(name='Jakob',role='Dev')
db.session.add(sev)
db.session.add(jakob)
db.session.commit()

In [ ]:
Project_member.query.all()

SQL Alchemy now runs a background query to fetch all tasks associated with a queried user:

In [ ]:
jakob=Project_member.query.filter_by(name='Jakob').first()
jakob.tasks

After adding some tasks, and assigning them to the PK of Jakob, they will show up on the next query:

In [ ]:
task_1=Task(name='Implement Flask App',owner_id=jakob.id)
task_2=Task(name='Document Learnings',owner_id=jakob.id)
db.session.add(task_1)
db.session.add(task_2)
db.session.commit()
Project_member.query.filter_by(name='Jakob').first().tasks

This is it for now, so let's drop our tables again to make sure the DB is in a clean state:

In [ ]:
db.drop_all()